In [15]:
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import time
import pickle as pl

# custom imports
from get_vocabulary import get_vocab
from encoder_decoder import Encoder, Decoder


print(f"tf=={tf.__version__}")
print(f"keras=={keras.__version__}")
print(f"np=={np.__version__}")
print(f"pd=={pd.__version__}")

tf==2.15.0
keras==2.15.0
np==1.26.2
pd==2.1.4


In [16]:
MAX_LINES = 40_000
MAX_WORDS = 11
units = 1024    # dimensionality of output of RNN

In [17]:
with open(file="../../trained_model/src_lang_tokenizer.pkl", mode="rb") as f:
    src_lang_tokenizer: keras.preprocessing.text.Tokenizer = pl.load(f)
with open(file="../../trained_model/tgt_lang_tokenizer.pkl", mode="rb") as f:
    tgt_lang_tokenizer: keras.preprocessing.text.Tokenizer = pl.load(f)


In [18]:
def translate(sentence, encoder, decoder):
    # attention_plot = np.zeros((max_length_targ, max_length_inp))
    # sentence = preprocess_sentence(sentence)

    inputs = [src_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],maxlen=MAX_WORDS, padding='post')

    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1,units))]
    _, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([tgt_lang_tokenizer.word_index['<start>']], 0)

    for t in range(MAX_WORDS):
        predictions, dec_hidden = decoder(dec_input, dec_hidden)
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += tgt_lang_tokenizer.index_word[predicted_id] + ' '

        if tgt_lang_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence
        
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence

In [19]:
encoder_save_path = "../../trained_model/seq2seq-trained-encoder"
decoder_save_path = "../../trained_model/seq2seq-trained-decoder"
encoder = keras.models.load_model(filepath=encoder_save_path)
decoder = keras.models.load_model(filepath=decoder_save_path)

In [20]:
result, sentence = translate(u'<start> debatten forklarar jag avslutad . <end>', encoder, decoder)
print(f'Input sentence  : {sentence}')
print(f'Translation     : {result}')


ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (2 total):
    * <tf.Tensor 'x:0' shape=(1, 11) dtype=int32>
    * [<tf.Tensor 'hidden:0' shape=(1, 1024) dtype=float32>]
  Keyword arguments: {'training': False}

 Expected these arguments to match one of the following 2 option(s):

Option 1:
  Positional arguments (2 total):
    * TensorSpec(shape=(None, 11), dtype=tf.int32, name='x')
    * TensorSpec(shape=(None, 1024), dtype=tf.float32, name='hidden')
  Keyword arguments: {'training': True}

Option 2:
  Positional arguments (2 total):
    * TensorSpec(shape=(None, 11), dtype=tf.int32, name='x')
    * TensorSpec(shape=(None, 1024), dtype=tf.float32, name='hidden')
  Keyword arguments: {'training': False}

In [ ]:
"jag skulle vilja ge dem till belgien"
"en man ska laga mat i ett hem"
"det tycka jag vara helt oacceptabelt ."



result, sentence = translate(
    u"<start> det tycker jag ar helt oacceptabelt . <end>",          # jag kommer inte att satta debatten om ja
    encoder, 
    decoder
    )
print('Input: %s' % (sentence))
print('Predicted translation: {}'.format(result))